# SageMaker Extension initialization

Here we will perform all the steps to get the SageMaker Extension functionality up and running. Please refer to the SageMaker Extension <a href="https://github.com/exasol/sagemaker-extension/blob/main/doc/user_guide/user_guide.md" target="_blank" rel="noopener">User Guide</a> for details on the required initialization steps. The extension module should have already been installed during the installation of this product, therefore the first step mentioned in the guide can be skipped.

We will be running SQL queries using <a href="https://jupysql.ploomber.io/en/latest/quick-start.html" target="_blank" rel="noopener"> JupySQL</a> SQL Magic and <a href="https://github.com/exasol/pyexasol" target="_blank" rel="noopener">`pyexasol`</a> module.

## Prerequisites

Before using this notebook the following steps need to be completed:
1. [Configure the sandbox](../sandbox_config.ipynb).

## Setup

### Access configuration

In [ ]:
%run ../utils/access_store_ui.ipynb
display(get_access_store_ui('../'))

### Set AWS credentials

We need to enter security credentials that will allow us to use the AWS services.

In [ ]:
%run utils/sme_init_ui.ipynb
display(get_sme_config_ui())

## Initialize the extension

<b>This operation normally takes a considerable amount of time to complete</b>

When the initialization finishes we should see a printed output suggesting us to activate the language container. Let's take it as an indication of the successful completion of the initialization procedure. We are not going to do the language container activation manually.

In [ ]:
from exasol.sagemaker_extension_wrapper import initialize_sme_extension

initialize_sme_extension(sb_config)

Now we are ready to start training a model. We will do this in the [following](sme_train_model.ipynb) notebook.